Discover Mechanics
==========

Let's do some Monte Carlos for Discover as well.  Some ones to try:

* Tomb Spider (Discover a Beast)
* Jewled Scarab (Discover a 3 Cost Card)
* Dark Peddler (Discover a 1-Cost Card - Warlock Only)
* Ethereal Conjurer (Discover a Spell - Mage Only)

In [4]:
# Load up all the cards
from helpers.data import load_dataframe, player_classes
from helpers.discover import discover
from collections import Counter
df = load_dataframe()

In [7]:
# Tomb Spider (Discover a Beast)
# First, let's filter down to just the beasts
beasts = df[df.race == "Beast"]

# let's get the initial optimized dataframes for each class
beasts_by_class = dict()
for cls in player_classes:
    discarded_best, discarded_picks, optimized_df = discover(cls, beasts)
    beasts_by_class[cls] = optimized_df
    
# our function for monte-carloing this
def tomb_spider_monte_carlo(cls, n=10000):
    best_counter = Counter()
    picks_counter = Counter()
    for i in range(n):
        best, picks, discard = discover(cls, beasts_by_class[cls], needs_weights=False)
        best_counter.update([best["name"]])
        picks_counter.update(picks["name"].values)
    return best_counter, picks_counter

In [14]:
best, picks = tomb_spider_monte_carlo("Hunter", n=1000)
best.most_common(30)

[(u'Savannah Highmane', 146),
 (u'King Krush', 127),
 (u"Gahz'rilla", 119),
 (u"King's Elekk", 93),
 (u'Dreadscale', 77),
 (u'Webspinner', 64),
 (u'Stampeding Kodo', 37),
 (u'Haunted Creeper', 32),
 (u'Stranglethorn Tiger', 27),
 (u'Core Rager', 26),
 (u'Scavenging Hyena', 23),
 (u'Tundra Rhino', 20),
 (u'Jeweled Scarab', 19),
 (u'Maexxna', 18),
 (u'King of Beasts', 16),
 (u'Jungle Panther', 15),
 (u'King Mukla', 13),
 (u'Captured Jormungar', 13),
 (u'Armored Warhorse', 12),
 (u'Emperor Cobra', 12),
 (u'Tomb Spider', 11),
 (u'Lost Tallstrider', 10),
 (u'Bloodfen Raptor', 9),
 (u'River Crocolisk', 9),
 (u'Ironbeak Owl', 8),
 (u'The Beast', 8),
 (u'Timber Wolf', 7),
 (u'Dire Wolf Alpha', 7),
 (u'Ironfur Grizzly', 5),
 (u'Oasis Snapjaw', 5)]